# Stochastic Simulation: Discrete Event Simulation

In [1]:
import random as rd
import TidySimStat as tss

`TidySimStat.py` by Edward J. Xu is imported. Copyright all reserved. 
Last modifed date is June 2, 2020.


In [41]:

class Node:

    def __init__(self, index):
        self._index = index
        self._next = None


class Head:

    def __init__(self):
        "Initiate an empty linked list."
        self._next = None

    def insert(self, node_new):
        "Insert the given node according to indices."
        if self.last._index <= node_new._index:
            ## If the index of the last node is smaller than the index of the
            ## given node, there is no need to check all nodes.
            self.last._next = node_new
        else:
            cur = self._next
            while True:
                if cur._next._index < node_new._index:
                    cur = cur._next
                else:
                    node_new._next = cur._next
                    cur._next = node_new
                    break

    def undock():
        """Undock the first node and return it. The second node become the
        first one."""
        undocked = self._next
        self._next = self._next._next

        return undocked

    def collect_index(self):
        cur = self._next
        indices = {}
        i = 0
        while cur:
            indices[i] = cur._index
            cur = cur._next
            i += 1
        n = len(indices)
        print(n)
        results = [indices[i] for i in range(n)]


class Arrival(Node):

    def __init__(self, time):
        super().__init__(time)
        self.whe_block = None
        self._next_arrived = None


class Leave(Node):

    def __init__(self, time:float, whi_server:int):
        super().__init__(time)
        self.whi_server = whi_server
        self._arrival = None


class Servers(Head):

    def __init__(self, f_serve, f_arrive, num:int=5):
        """Queueing system modelled by linked lists.

        Notes
        =====
        There is no waiting room in this setting.
        """
        if not callable(f_serve):
            raise ValueError("Function to simulate service time is "
                "not callable.")
        if not callable(f_arrive):
            raise ValueError("Function to simulate arrival sojourn time is "
                "not callable.")

        super().__init__()
        self.states = [0 for i in range(num)]
        self.num_arrival = 0
        self.num_block = 0
        self.f_serve = f_serve
        self.f_arrive = f_arrive
        self.clock = 0
        self._next_arrived = None

        self.warmup()

    def warmup(self):
        """Warm up the scheduled arrivals.

        Notes
        =====
        There is no need to schedule multiple arrivals in warming up stage.
        """
        t = self.f_arrive()
        self._next = Arrival(t)
        # self.states[0] = 1
        # self._next_arrived = self._next

    def schedule_arrival(self):
        """The new arrival always follows the last scheduled arrival."""
        t = self._next._index + self.f_arrive()
        self.insert(Arrival(t))

    def schedule_leave(self, whi_server):
        t = self.clock + self.f_serve()
        self.insert(Leave(t, whi_server))

    def arrive(self):
        "Event routine triggered when a new customer arrives."
        self.num_arrival += 1
        whi_server = self.first_idle + 0
        if whi_server > self.num:  # There is no idle server.
            ## If the customer is blocked, there is no need to set a
            ## `leave` event.
            self.num_block += 1
            self._next.whe_block = 1
        else:
            ## To assign the customer to the first idle server and simulate
            ## his/her leaving time.
            self.states[whi_server] = 1
            # print(self.states)
            self.schedule_leave(whi_server)

        ## Next schedule
        self.schedule_arrival()

    def leave(self):
        "Event routine triggered when an existing customer leaves."
        self.states[self._next.whi_server] = 0  # To set the server idle.

    def advance(self):
        "Invoke next event and advance the clock time."
        self.clock = self._next._index + 0

        if type(self._next) is Leave:
            self.leave()
            self._next = self._next._next
        else:
            self.arrive()
            self._last_schedule = self._next
            self._next = self._next._next

        return self.clock

    @property
    def num(self):
        return len(self.states)

    @property
    def first_idle(self):
        result = 0
        i = 0
        while i <= self.num:
            if self.states[i] == 0:
                break
            else:
                i += 1
        return i

    @property
    def last(self):
        cur = self._next
        while cur._next:
            cur = cur._next
        return cur

    # @property
    # def last_arrival(self):
    #     cur = self._next_arrived
    #     while cur._next_arrived:
    #         cur = cur._next_arrived
    #     return cur


In [42]:
ser = Servers(lambda: 5, lambda: 5, 10)
ser.collect_index()

1


In [43]:
[1] == [1]

True

In [34]:
ser = Servers(num=10, f_serve=lambda: tss.sim_exp(5), f_arrive=lambda: tss.sim_exp(5))

In [35]:
ser.warmup()
ser.show_all()

AttributeError: 'Servers' object has no attribute '_last_schedule'

In [22]:
ser = Servers(num=10, f_serve=lambda: tss.sim_exp(5), f_arrive=lambda: tss.sim_exp(5))
ser.warmup()
clock = 0
while clock <= 100:
    clock = ser.advance()

[1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
[1, 1, 0, 1, 0, 0, 0, 0, 0, 0]
[1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 0, 0, 0, 0, 0, 0]


In [24]:
ser.show_all()

1-th event invoke time: 101.4041845697292
2-th event invoke time: 103.3489731418054
3-th event invoke time: 106.26527470019144
4-th event invoke time: 111.51176292771603
5-th event invoke time: 111.62918458882754
6-th event invoke time: 116.43794989568627
7-th event invoke time: 117.27897965288827
8-th event invoke time: 123.48802146860986
9-th event invoke time: 126.09902556415422
10-th event invoke time: 136.0048862193945
11-th event invoke time: 137.12271299637982
12-th event invoke time: 139.71482334447103
13-th event invoke time: 141.4408838512817
14-th event invoke time: 146.45844764868093


In [23]:
ser.schedule()
ser.last._index

146.45844764868093

In [32]:
type(ser.last) is Event

True

In [30]:
def add(num, nums=[]):
#     if nums is None:
#         nums = []
    nums.append(num)
    print(nums)

In [31]:
print(add.__defaults__)
add(1)
print(add.__defaults__)

([],)
[1]
([1],)
